<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#导入相关的包" data-toc-modified-id="导入相关的包-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入相关的包</a></span></li><li><span><a href="#导入数据" data-toc-modified-id="导入数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入数据</a></span></li><li><span><a href="#形成训练数据" data-toc-modified-id="形成训练数据-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>形成训练数据</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>训练模型</a></span></li></ul></div>

# 导入相关的包

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

D:\ProgramAPP\python\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 导入数据

In [2]:
%%time
trains = pd.read_csv('../raw_data/d_train.csv',encoding="gbk")
tests = pd.read_csv("../raw_data/d_test_B_20180128.csv",encoding="gbk")
trains.drop(trains[trains["年龄"] >= 84].index,inplace=True)
fea_train = pd.read_csv("../raw_data/fea_train.csv")
fea_test = pd.read_csv("../raw_data/fea_test_B.csv")
fea_train1 = pd.read_csv("../raw_data/fea_train_1.csv")
fea_test1 = pd.read_csv("../raw_data/fea_test_B_1.csv")
fea_train2 = pd.read_csv("../raw_data/fea_train_2.csv")
fea_test2 = pd.read_csv("../raw_data/fea_test_B_2.csv")
trains = pd.merge(trains, fea_train, how="left",on="id")
trains = pd.merge(trains, fea_train1, how="left",on="id")
# trains = pd.merge(trains, fea_train2, how="left",on="id")
tests = pd.merge(tests, fea_test, how="left",on="id")
tests = pd.merge(tests, fea_test1, how="left",on="id")
# tests = pd.merge(tests, fea_test2, how="left",on="id")
trains["血糖"] = trains["血糖"].apply(lambda x: 1 if x > 10 else 0)

Wall time: 480 ms


# 形成训练数据


In [3]:
%%time
def make_feat(train,test):
    train_id = train.id.values.copy()
    test_id = test.id.values.copy()
    data = pd.concat([train,test])
    data['性别'] = data['性别'].map({'男': 1,'女': 0})
    # data["性别"] = data['性别'].astype(int)
    # data['date'] = (pd.to_datetime(data['date']) - parse('2017-10-09')).dt.days
    # data['体检日期'] = (pd.to_datetime(data['体检日期']) - parse('2017-10-09')).dt.days
    data.drop("体检日期",axis = 1,inplace= True)
    # data.fillna(value = -1, inplace = True)
    train_feat = data[data.id.isin(train_id)]
    test_feat = data[data.id.isin(test_id)]
    return train_feat,test_feat
train, test = make_feat(trains, tests)
predictors = [f for f in list(train.columns) if f not in ["血糖","blood_sugar","id","blood_sugar_log","体检日期"]]
# X_train, X_test, y_train,y_test = train_test_split(train[predictors], train["血糖"],test_size=0.1,random_state=42)
xgb_train = xgb.DMatrix(train[predictors], label=train["血糖"])
# xgb_eval = xgb.DMatrix(X_test, label=y_test)
xgb_test = xgb.DMatrix(test[predictors])

Wall time: 189 ms


In [4]:
train.shape
test.shape
test.head()
train.head()
(len(train["血糖"]) - sum(train["血糖"])) / sum(train["血糖"])

39.73913043478261

# 训练模型

In [5]:
params={'booster':'gbtree',
    'objective': 'binary:logistic',
    'scale_pos_weight':float(len(train["血糖"]) - sum(train["血糖"])) / sum(train["血糖"]),
    'eval_metric': 'auc',
    "subsample":0.65,
    "colsample_bytree":0.8,
    "lambda":5,
    "seed":1024,
    "silent":1,
    "verbose":0,
    "max_depth":7,
    "alpha":0.01,
    "gamma":0.6
    }

In [6]:
%%time
watchlist  = [(xgb_train,'train')]
#通过cv找最佳的nround
cv_log = xgb.cv(params,xgb_train,num_boost_round=25000,nfold=5,metrics='auc',early_stopping_rounds=50,seed=1024)

Wall time: 13.4 s


In [7]:
cv_log

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.697459,0.018642,0.870281,3.841793e-03
1,0.712837,0.024899,0.934310,8.458303e-03
2,0.736680,0.021641,0.961428,4.915283e-03
3,0.748327,0.020912,0.974318,4.140214e-03
4,0.757033,0.023967,0.982408,2.975798e-03
5,0.768477,0.026678,0.987430,4.358138e-03
6,0.768646,0.042546,0.991238,2.363835e-03
7,0.774534,0.040099,0.993251,1.749517e-03
8,0.786044,0.027745,0.994933,1.053628e-03
9,0.795614,0.027617,0.996410,9.711621e-04


In [8]:
bst_auc= cv_log['test-auc-mean'].max()
cv_log['nb'] = cv_log.index
cv_log.index = cv_log['test-auc-mean']
bst_nb = cv_log.nb.to_dict()[bst_auc]

In [9]:
%%time
#train
watchlist  = [(xgb_train,'train')]
model = xgb.train(params,xgb_train,num_boost_round=bst_nb+50,evals=watchlist)

[0]	train-auc:0.882024
[1]	train-auc:0.931939
[2]	train-auc:0.96088
[3]	train-auc:0.97367
[4]	train-auc:0.975865
[5]	train-auc:0.980044
[6]	train-auc:0.988344
[7]	train-auc:0.991056
[8]	train-auc:0.992981
[9]	train-auc:0.996152
[10]	train-auc:0.996596
[11]	train-auc:0.997657
[12]	train-auc:0.998029
[13]	train-auc:0.998293
[14]	train-auc:0.998799
[15]	train-auc:0.999061
[16]	train-auc:0.999264
[17]	train-auc:0.999503
[18]	train-auc:0.999651
[19]	train-auc:0.999711
[20]	train-auc:0.999819
[21]	train-auc:0.999911
[22]	train-auc:0.999925
[23]	train-auc:0.999947
[24]	train-auc:0.999966
[25]	train-auc:0.999976
[26]	train-auc:0.999982
[27]	train-auc:0.999984
[28]	train-auc:0.999993
[29]	train-auc:0.999995
[30]	train-auc:0.999997
[31]	train-auc:0.999999
[32]	train-auc:1
[33]	train-auc:1
[34]	train-auc:1
[35]	train-auc:1
[36]	train-auc:1
[37]	train-auc:1
[38]	train-auc:1
[39]	train-auc:1
[40]	train-auc:1
[41]	train-auc:1
[42]	train-auc:1
[43]	train-auc:1
[44]	train-auc:1
[45]	train-auc:1
[46]	t

In [10]:
test_pred = model.predict(xgb_test)

In [11]:
test_result = test
test_result["gt10_prob"] = test_pred
# test_result.to_csv("../classifier/gt98_prob.csv",index=None,encoding='utf-8')

In [12]:
test_result[["id","gt10_prob"]].to_csv("../classification/result/gt10.csv",index=False)
test_result[test_result.gt10_prob.values > 0.499]["gt10_prob"].count()

9